In [1]:
import tensorflow as tf
import os
import numpy as np

## Load Model

In [2]:
model = tf.saved_model.load('translator')

In [3]:
result = model.translate(['sugar, salt, eggs, vanilla'], ['cake'])
result[0].numpy().decode()

'cake mix cake mix eggs and oil in a large bowl step beat in eggs one at a time beating well after each addition step add flour mixture and beat until combined step pour into prepared pan step bake at 350 degrees for 25 minutes or until cake tests done'